In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import csv
import os
import tensorflow as tf
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
# from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from test_utils import summary, comparator
# import public_tests

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#path='/kaggle/input/aptos2019-blindness-detection/train_images'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-09-30 08:25:40.872424: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def Read_training_data():
    path_training_imgs='/kaggle/input/aptos2019-blindness-detection/train_images'
    path_training_lbls='/kaggle/input/aptos2019-blindness-detection/train.csv'
    train_image_list=np.zeros((3662,256,256,3))
    train_lbls_list=np.zeros((1,3662))
    i=0
    j=0

    with open(path_training_lbls, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    if(row[1]=='diagnosis'):
                        continue
                    train_lbls_list[0][j]=int(row[1])
                    j=j+1
    Y_train =tf.keras.utils.to_categorical(train_lbls_list, num_classes=5, dtype="float32")
    print(Y_train.shape)
    for dirname, _, filenames in os.walk(path_training_imgs):
        for filename in filenames:
            image_path=os.path.join(dirname, filename)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img,(256,256))
#             img=np.asarray(img)
#             img=img.reshape(224,224,3)
            train_image_list[i]=img
            i=i+1

#     print(training_image_list.shape)
#     for dirname, _, filenames in os.walk(path_training_lbls):
    return train_image_list , Y_train

In [3]:
X, X_lbls=Read_training_data()
print(X.shape, X_lbls.shape)

(1, 3662, 5)
(3662, 256, 256, 3) (1, 3662, 5)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D , GaussianDropout
from tensorflow.keras import regularizers, optimizers
k=4
s=2
model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=16,kernel_size=(k,k),padding="valid"))
model.add(Conv2D(filters = 16,kernel_size=(k,k),strides=(s,s),padding = 'valid', activation ='relu', input_shape = (224,224,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(s,s)))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same'))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same' ))
model.add(Conv2D(filters = 64, kernel_size=(k,k),strides=(s,s),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(s,s)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dense(5, activation = "softmax"))

2021-09-30 08:32:12.649626: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-30 08:32:12.655114: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-30 08:32:12.699476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 08:32:12.700129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-30 08:32:12.700185: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-30 08:32:12.725951: I tensorflow/stream_executor/platform/def

In [5]:
from tensorflow.keras.optimizers import RMSprop,Adam
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999,amsgrad=True,clipnorm=1.,clipvalue=0.5)

In [6]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 253, 253, 16)      784       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 16)      4112      
_________________________________________________________________
batch_normalization (BatchNo (None, 125, 125, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        16448     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        65600     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          6

In [8]:
epochs = 20  # for better result increase the epochs
batch_size = 20

In [9]:
model.fit(X,X_lbls[0],batch_size=20, epochs=50)

2021-09-30 08:32:17.339886: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2879913984 exceeds 10% of free system memory.
2021-09-30 08:32:19.833323: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-30 08:32:19.846483: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000160000 Hz


Epoch 1/50


2021-09-30 08:32:21.465752: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-30 08:32:22.320558: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-30 08:32:22.360831: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


184/184 [==============================] - 13s 24ms/step - loss: 1.8336 - accuracy: 0.3272
Epoch 2/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3588 - accuracy: 0.4590
Epoch 3/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3301 - accuracy: 0.4715
Epoch 4/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3416 - accuracy: 0.4646
Epoch 5/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3169 - accuracy: 0.4882
Epoch 6/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3424 - accuracy: 0.4799
Epoch 7/50
184/184 [==============================] - 4s 23ms/step - loss: 1.3206 - accuracy: 0.4812
Epoch 8/50
184/184 [==============================] - 4s 23ms/step - loss: 1.3128 - accuracy: 0.4858
Epoch 9/50
184/184 [==============================] - 4s 24ms/step - loss: 1.3095 - accuracy: 0.4904
Epoch 10/50
184/184 [==============================] - 4s 23ms/step - loss: 1.3263 - accuracy: 0.4768

In [10]:
submit = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/sample_submission.csv')
predicted = []

In [11]:
from tqdm import tqdm
for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('/kaggle/input/aptos2019-blindness-detection/test_images/', name+'.png')
    image = cv2.imread(path)
    #print(path)
    image = cv2.resize(image, (256, 256))
    X = np.array((image[np.newaxis])/255)
   # print(X[0][150])
   # print(((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist())
    score_predict=model.predict(X)
    print(score_predict)
    label_predict = np.argmax(score_predict)
    predicted.append(str(label_predict))

3it [00:00,  6.40it/s]

[[0.84589154 0.08588351 0.05565479 0.00843078 0.00413938]]
[[0.84401673 0.08853997 0.05397263 0.00883671 0.004634  ]]
[[0.8381257  0.09218507 0.05545624 0.00947387 0.00475913]]
[[0.8420061  0.08994204 0.05504293 0.00897621 0.00403267]]


5it [00:00,  9.60it/s]

[[0.84595454 0.08540946 0.05500462 0.00898303 0.00464826]]
[[0.85413027 0.08051842 0.05269084 0.00830717 0.00435336]]


9it [00:00, 11.46it/s]

...

[[0.84301096 0.08952072 0.05310059 0.00966258 0.00470512]]
[[0.8488904  0.08715153 0.05084936 0.00881756 0.00429107]]
[[0.8509533  0.08108965 0.05459454 0.0089758  0.00438677]]
[[0.8495719  0.08538707 0.0513597  0.00880347 0.0048779 ]]


1926it [02:30, 13.78it/s]

[[0.83647585 0.09060217 0.05881663 0.00968244 0.0044229 ]]
[[0.8482308  0.08351304 0.05562611 0.0085934  0.00403673]]


1928it [02:30, 12.82it/s]

[[0.84648997 0.08655566 0.0522999  0.00975384 0.00490053]]
[[0.8248166  0.09316    0.06640652 0.01100543 0.00461149]]


In [12]:
submit['diagnosis'] = predicted
submit.to_csv('submission.csv', index=False)
submit.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [13]:
print(predicted)

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',